# Homework 4

I said earlier in the course that I wouldn't use Jupyter beyond a demo.  I lied.  Sorry.

We'll see how this goes!



## Problem 0


We talked about the challenging demon child that is $0.1_{10}$.  Show me in any way that you can that this is an impossible number to store exactly in 64-bit precision (aka, a Python `float`).  (Hint: "series" from Calc 2).

*YOUR WORDS HERE INSTEAD OF MINE!*

LaTeX is wonderful.  For example, I can write in nice, pretty notation that

$$
1 + \frac{1}{2} + \frac{1}{4} + \ldots = \sum^{\infty}_{i=0} 2^{-i} = 2 \, .
$$

I can show (bit not prove) this result numerically via

In [ ]:
# and/or YOUR CODE HERE INSTEAD OF MINE!

i, s = 0, 0
while abs(2**(-i)) > 3e-16:  # remember 2**(-52), right?
    s = s + 2**(-i)
    i = i + 1
print(s)

**Replace my text and code above.**

## Problem 1

I can quickly make a matrix with a structure similar to the one we saw for the 2d heat conduction problem for a square domain with an **even** number of equal-sized cells along each axis:

In [ ]:
import numpy as np
L = 1.0  # length of a side (m)
k = 1.0  # thermal conductivity (W/m/C)
n = 10  # number of cells... make me stay EVEN!!
assert  n % 2 == 0, "n has to be even, yo"
A = np.diag(-np.ones(n), 0) + \
    np.diag(0.25*np.ones(n-1), k=-1) + \
    np.diag(0.25*np.ones(n-1), k= 1) + \
    np.diag(0.25*np.ones(n//2), k=-n//2) + \
    np.diag(0.25*np.ones(n//2), k= n//2)
A = A * k / (L/n)**2

In [ ]:
plt.spy(A)

Now, suppose the heat source is uniform and equal to 1 W/m.  In other words, the right-hand side is a bunch of ones.  

**Part A** Show me the temperature as a function of $x$ and $y$ for $n = 10$.  You may adapt the plotting code provided in `FD2D_HEAT_STEADY`.

In [ ]:
# YOUR CODE HERE

**Part B** Show me how long it takes to solve for $n = 10, 100, 1000, 10000, 100000$.  Make your "evidence" a plot (good plotting practices apply; when in doubt, use [best practices](https://robertsj.github.io/me400_notes/lectures/Basic_Data_Processing_with_NumPy_and_Matplotlib.html#Diving-into-pyplot). You'll need `import time`, etc., to do this problem.  Look to Rohan's examples for ways to time a function call.

In [ ]:
# YOUR CODE HERE

## Problem 2

Any "dense" matrix can be converted to a (compressed-storage-row) sparse matrix in Python using the following:

In [ ]:
import scipy.sparse as sparse
A_sparse = sparse.csr_matrix(A)
A_sparse

Use this to reproduce your **Part B** to Problem 1 using `sparse.gmres` instead of `np.linalg.solve`.

In [ ]:
# YOUR CODE HERE

Just know this is *not* where this problem could end.  Preconditioners, convergence criteria, and more are things you can explore further if you end up in a world where big, sparse systems represent a recurring theme.

## Problem 4

In ME 535, I just made measurements to calibrate a variable potentiometer.  Below are the number of rotations and the resulting voltage (mV).

```
turns  voltage
--------------
1	   11.6	
2	   24.8	 
3	   37.0	 
4	   49.1 
5	   60.9 
6	   72.8 	
7	   84.5 		
8	   96.3 		
9	  107.7		
```

**Part A**

Use `np.polyfit` and `np.polyval` to fit this data to a line, i.e., $v = at + b$, where $v$ is voltage, $t$ is turns, $a$ is the slope$, and $b$ is the intercept.  Then, plot the data (using points) and the fit (as a line) on one figure.

In [ ]:
# here's the data so you don't have to type it
turns = np.array([1,2,3,4,5,6,7,8,9.0]) # 9.0 to force this to be floats and not ints!
voltage = np.array([11.6, 24.8,37.0,49.1 ,60.9 ,72.8, 84.5, 96.3, 107.7])

In [ ]:
## YOUR CODE HERE
np.polyfit(turns, voltage, 2)

**Part B**

Use the same procedure to fit the model $v = at^2 + bt + c$.  Careful: this $a$ and $b$ are *different* from **Part A**, so choose wise Python variable names.  Plot the data and both fits (linear and quadratic) on one figure.  

In [ ]:
## YOUR CODE HERE

**Part C**

For arbitrary models of the form $v = f(t, a, b, c, \ldots)$, we can use `curve_fit`.  The most basic use of `curve_fit` looks like

```python
curve_fit(f, turns, voltages, (a0, b0, c0))
```

where `f` is a Python function that represents the model, e.g.,

```
def f(t, a, b, c):
    return a*t**2 + b*t + c
```

and `a0`, `b0`, and `c0` are initial guesses for the coefficients.  Here, use `curve_fit` with `a0=0`, `b0=12`, and `c0=0`.  Report the computed values for `a`, `b`, and `c`.  

In [ ]:
from scipy.optimize import curve_fit

def f(t, a, b, c):
    return a*t**2 + b*t + c

a0, b0, c0 = 0.0, 12.0, 0.0

## YOUR CODE HERE

**Part D**

Yet another way to perform fitting is to apply minimization directly to error of a fit.  Everything we've done above is based on "least squares".  In other words, if $v$ is the measured voltage and $\tilde{v}$ is the *approximate* voltage, we found $a$, $b$, and maybe $c$ that minimize the sum of the square errors$^*$:

$$
  \min_{a,b,c} \sqrt{  \frac{1}{10} \sum_{i=1}^{10} \Big (v_i - \tilde{v}_i(a,b,c) \Big)^2 }
$$

$*$Note that the $\sqrt{\cdot}$ and $1/10$ make this the "root, mean" square error, but the result is the same...think about why!


Run this little bit of code to show that `minimize` yields the same result as **Part B** and **Part C**.

In [ ]:
from scipy.optimize import minimize

def objective_L2(coefs, t, v):
    a, b, c = coefs
    v_tilde = a*t**2 + b*t + c # or use f(t, a, b, c)
    error = v - v_tilde
    sum_square_error =  np.sum(error**2)
    return sum_square_error

minimize(objective, x0=(a0, b0, c0), args=(turns, voltage))

Least squares is usually "easy", and it has a strong connection to statistics and the normal distribution, but it isn't the only way to fit a model.  Note that by minimizing the *square* of the error, the fitting process naturally penalizes the biggest errors most. What if we instead want to penalize all errors *equally*?  The right objective then is to minimize this quantity:

$$
  \min_{a,b,c} \frac{1}{10} \sum_{i=1}^{10} |v_i - \tilde{v}_i(a,b,c) |
$$

where $|\cdot|$ is the absolute value. By minimizing this quantity, the *average* magnitude of the errors is minimized.  Define a new `objective_L1` function that fits the data using this approach.

In [ ]:
## YOUR CODE HERE

**Part E**

Finally, show me the mean, square error and the mean absolute error for your fits from **Part C** and **Part D**.

In [ ]:
## YOUR CODE HERE

## Problem 5

Go find any ordinary differential equation that still haunts you from MATH 340
and solve it numerically using `odeint`.  

**STATE THE ODE PROBLEM HERE**

In [ ]:
from scipy.integrate import odeint

## YOUR CODE HERE